# Initialisation

In [1]:
root_dir = "/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/"

## General imports

In [2]:
import io
import csv
import os.path

import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

In [3]:
%matplotlib qt5

In [4]:
import sys

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.ekfnavtools import *
import posarmctools.epsgtools as epsg

In [6]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.PosarMCParameters import *

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
withPlots = 0

## EPSG transformations

In [9]:
import pyproj

In [10]:
# Define some common projections using EPSG codes
wgs84 = pyproj.Proj("+init=EPSG:4326")      # LatLon with WGS84 datum used by GPS units and Google Earth
epsg3857 = pyproj.Proj("+init=EPSG:3857")   # WGS84 / Pseudo Mercator
epsg3395 = pyproj.Proj("+init=EPSG:3395")   # WGS84 / World Mercator
epsg3948 = pyproj.Proj("+init=EPSG:3948")   # RGF93 / CC48 Projected coordinate system
epsg32630 = pyproj.Proj("+init=EPSG:32630") # WGS 84 / UTM zone 30N

epsg3xxx = epsg32630 # epsg3857 does not seem to keep distances

In [11]:
if epsg3xxx == epsg3857:
    epsgStr = "epsg3857"
    
if epsg3xxx == epsg3948:
    epsgStr = "epsg3948"
    
if epsg3xxx == epsg32630:
    epsgStr = "epsg32630"

## Scene remarkable points

In [12]:
corner = [ 48 + 03.462/60, -( 2 + 00.507/60 ) ]
corner_epsg = epsg.wgs84ToEpsg( corner[::-1], epsg3xxx )

h0 = [ 48 + 03.409/60, -( 2 + 00.468/60 ) ]
h1 = [ 48 + 03.409/60, -( 2 + 00.481/60 ) ]
h2 = [ 48 + 03.401/60, -( 2 + 00.479/60 ) ]
h3 = [ 48 + 03.401/60, -( 2 + 00.467/60 ) ]
h0_epsg = epsg.wgs84ToEpsg( h0[::-1], epsg3xxx )
h1_epsg = epsg.wgs84ToEpsg( h1[::-1], epsg3xxx )
h2_epsg = epsg.wgs84ToEpsg( h2[::-1], epsg3xxx )
h3_epsg = epsg.wgs84ToEpsg( h3[::-1], epsg3xxx )

p0 = [ 48 + 03.453/60, -( 2 + 00.629/60 ) ]
p1 = [ 48 + 03.440/60, -( 2 + 00.624/60 ) ]
p2 = [ 48 + 03.504/60, -( 2 + 00.358/60 ) ]
p3 = [ 48 + 03.491/60, -( 2 + 00.353/60 ) ]
p0_epsg = epsg.wgs84ToEpsg( p0[::-1], epsg3xxx )
p1_epsg = epsg.wgs84ToEpsg( p1[::-1], epsg3xxx )
p2_epsg = epsg.wgs84ToEpsg( p2[::-1], epsg3xxx )
p3_epsg = epsg.wgs84ToEpsg( p3[::-1], epsg3xxx )

s0 = [ 48 + 03.649/60, -( 1 + 59.701/60 ) ] # 82 m
s1 = [ 48 + 03.650/60, -( 2 + 01.313/60 ) ] # 98 m
s2 = [ 48 + 03.274/60, -( 2 + 01.354/60 ) ] # 107 m
s3 = [ 48 + 03.275/60, -( 1 + 59.725/60 ) ] # 84 m
s4 = [ 48 + 04.003/60, -( 2 + 00.793/60 ) ] # 96 m
s5 = [ 48 + 02.940/60, -( 2 + 00.793/60 ) ] # 77 m
s6 = [ 48 + 02.916/60, -( 2 + 00.228/60 ) ] # 83 m
s7 = [ 48 + 04.015/60, -( 2 + 00.226/60 ) ] # 91 m

s0_epsg = epsg.wgs84ToEpsg( s0[::-1], epsg3xxx )
s1_epsg = epsg.wgs84ToEpsg( s1[::-1], epsg3xxx )
s2_epsg = epsg.wgs84ToEpsg( s2[::-1], epsg3xxx )
s3_epsg = epsg.wgs84ToEpsg( s3[::-1], epsg3xxx )
s4_epsg = epsg.wgs84ToEpsg( s4[::-1], epsg3xxx )
s5_epsg = epsg.wgs84ToEpsg( s5[::-1], epsg3xxx )
s6_epsg = epsg.wgs84ToEpsg( s6[::-1], epsg3xxx )
s7_epsg = epsg.wgs84ToEpsg( s7[::-1], epsg3xxx )

x0 = [ 48 + 03.661/60, -( 1 + 58.893/60 ) ]
x1 = [ 48 + 03.647/60, -( 2 + 02.121/60 ) ]
x2 = [ 48 + 03.273/60, -( 2 + 02.141/60 ) ]
x3 = [ 48 + 03.278/60, -( 1 + 58.933/60 ) ]
x4 = [ 48 + 04.572/60, -( 2 + 00.798/60 ) ]
x5 = [ 48 + 02.410/60, -( 2 + 00.795/60 ) ]
x6 = [ 48 + 02.394/60, -( 2 + 00.226/60 ) ]
x7 = [ 48 + 04.558/60, -( 2 + 00.229/60 ) ]

np.save( f"{root_dir}corner_epsg", corner_epsg )
np.save( f"{root_dir}hangar_epsg", (h0_epsg, h1_epsg, h2_epsg, h3_epsg) )
np.save( f"{root_dir}runaway_epsg", (p0_epsg, p1_epsg, p2_epsg, p3_epsg))
np.save( f"{root_dir}s_epsg", (s0_epsg, s1_epsg, s2_epsg, s3_epsg, s4_epsg, s5_epsg, s6_epsg, s7_epsg))

In [13]:
def addRemarkablePoints( ax ):
    ax.plot( s0_epsg[0], s0_epsg[1], 'oy', markeredgecolor = 'black' )
    ax.plot( s1_epsg[0], s1_epsg[1], 'oy', markeredgecolor = 'black' )
    ax.plot( s2_epsg[0], s2_epsg[1], 'oy', markeredgecolor = 'black' )
    ax.plot( s3_epsg[0], s3_epsg[1], 'oy', markeredgecolor = 'black' )
    ax.plot( s4_epsg[0], s4_epsg[1], 'oy', markeredgecolor = 'black' )
    ax.plot( s5_epsg[0], s5_epsg[1], 'oy', markeredgecolor = 'black' )
    ax.plot( s6_epsg[0], s6_epsg[1], 'oy', markeredgecolor = 'black' )
    ax.plot( s7_epsg[0], s7_epsg[1], 'oy', markeredgecolor = 'black' )
    ax.plot( corner_epsg[0], corner_epsg[1], 'og', markeredgecolor = 'black' )

# SBG data

## data path definitions

In [14]:
day = "2019_07_12"
prefix = "/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12_SBG/session_0003/" + day + "/"

hours = ["11h00", "12h00"]
nbHours = len(hours)

logEkfEuler = [ prefix + h + "/sbgLogEkfEuler.dat" for h in hours ]
logEkfNav  = [ prefix + h + "/sbgLogEkfNav.dat" for h in hours ]
logUtcData = [ prefix + h + "/sbgLogUtcData.dat" for h in hours ]
logGpsPos = [ prefix + h + "/sbgLogGpsPos.csv" for h in hours ]
logGpsVel = [ prefix + h + "/sbgLogGpsVel.dat" for h in hours ]
logEventB = [ prefix + h + "/sbgLogEventB.dat" for h in hours ]

##  logGpsPos

In [15]:
idx_gpsPos_status = 1
idx_gpsPos_lat = 3
idx_gpsPos_long = 4
idx_gpsPos_alt = 5
idx_gpsPos_undulation = 6
idx_gps_lat = 1
idx_gps_long = 2
idx_gps_alt = 3
idx_gps_undulation = 4
idx_gps_status = 5

def loadLogGpsPos( gps, n ):
    for idx in range(n):
        gps.append( np.loadtxt( logGpsPos[idx], skiprows = 1, delimiter = ',',
                               usecols = (0, idx_gpsPos_lat, idx_gpsPos_long, idx_gpsPos_alt, idx_gpsPos_undulation,
                                         idx_gpsPos_status) ) )

### Load data

In [16]:
gps=[]
loadLogGpsPos( gps, nbHours )

In [17]:
gps_all = gps[0]
for idx in range(1, 2):
    gps_all = np.concatenate( (gps_all, gps[idx]), axis=0 )

In [18]:
gpsTimestamps = gps_all[:, 0]
Lat = gps_all[:,idx_gps_lat]
Long = gps_all[:,idx_gps_long]
Alt = gps_all[:,idx_gps_alt]
Undulation = gps_all[:,idx_gps_undulation]
gpsPos_status = gps_all[:,idx_gps_alt].astype("uint32")

In [19]:
posStatus = np.zeros( gpsPos_status.shape )
posType = np.zeros( gpsPos_status.shape )
for idx, val in enumerate(gpsPos_status):
    posStatus[idx] = val & 0b000011
    posType[idx] = np.right_shift( (val & 0b111111000000), 6)

### Shift timestamps

In [20]:
for k in range(1,gpsTimestamps.shape[0]):
    if gpsTimestamps[k] < gpsTimestamps[k-1]:
        print("k = {}".format(k))

In [21]:
gpsNewTimestamps = gpsTimestamps

### Transform GPS data

In [22]:
idx_lat = 1
idx_long = 2

In [23]:
idx = 0
x0, y0 = epsg.wgs84ToEpsg( (gps[idx][:,idx_long], gps[idx][:,idx_lat]), epsg3xxx )

idx = 1
x1, y1 = epsg.wgs84ToEpsg( (gps[idx][:,idx_long], gps[idx][:,idx_lat]), epsg3xxx )

x = [x0, x1]
y = [y0, y1]

In [24]:
gps_proj = epsg.wgs84ToEpsg( (Long, Lat), epsg3xxx )
np.save( root_dir + "gps_epsg_transform", gps_proj )

## logGpsVel

### Load data

In [25]:
idx_gpsVel_status = 1
idx_gpsVel_north = 3
idx_gpsVel_east = 4
idx_gpsVel_down = 5
idx_gpsVel_course = 9

vel=[]
for idx in range(nbHours):
    vel.append( np.loadtxt( logGpsVel[idx], skiprows = 1,
                           usecols = (0, idx_gpsVel_north, idx_gpsVel_east, idx_gpsVel_down, 
                                      idx_gpsVel_course, idx_gpsVel_status) ) )

In [26]:
vel_all = vel[0]
for idx in range(1, nbHours):
    vel_all = np.concatenate( (vel_all, vel[idx]), axis=0 )
velNorth = vel_all[:,1]
velEast = vel_all[:,2]
velDown = vel_all[:,3]

In [27]:
Vel = ( velNorth**2 + velEast**2 + velDown**2) **0.5
course = vel_all[:,4]
velTimestamps = vel_all[:,0]
gpsVel_status = vel_all[:,5].astype("uint32")

## sbgLogEventB

### Load data

In [28]:
event = []
for idx in range(nbHours):
    event.append( np.loadtxt( logEventB[idx], skiprows = 1, usecols = 0 ) )

indexes = []
indexes.append( np.arange(event[0].size) )
for idx in range(1, nbHours):
    indexes.append( np.arange(event[idx].size)+indexes[idx-1][-1]+1)

In [29]:
if event[0].size == 1:
    event_all = np.array([event[0]])
else:
    event_all = event[0]
for idx in range(1, nbHours):
    event_all = np.concatenate( (event_all, event[idx]), axis=0 )

## logUtcData

### Load data

In [30]:
#timeStamp status year month day hour minute second nanoSecond gpsTimeOfWeek
idx_h = 5
idx_m = 6
idx_s = 7
idx_nano = 8

utc = []
for idx in range(nbHours):
    utc.append( np.loadtxt( logUtcData[idx], skiprows = 1, usecols = (0, idx_h, idx_m, idx_s, idx_nano) ) )

In [31]:
utc_all = utc[0]
for idx in range(1, nbHours):
    utc_all = np.concatenate( (utc_all, utc[idx]), axis=0 )

In [32]:
printUtc( 0, utc_all )
printUtc( 1, utc_all )
printUtc( -1, utc_all )

0.0 : 6.0 : 56.525
11.0 : 52.0 : 32.010
12.0 : 35.0 : 46.905


In [33]:
utcTimestamps = utc_all[:, 0]

### Shift timestamps

In [34]:
for k in range(1,utcTimestamps.shape[0]):
    if utcTimestamps[k] < utcTimestamps[k-1]:
        print("k = {}".format(k))

In [35]:
utcNewTimestamps = utcTimestamps

In [36]:
if 1:
    plt.figure()
    plt.plot( utcNewTimestamps, 'o' )
    plt.plot( utcTimestamps,'.' )
    plt.grid()

In [37]:
printUtc( 0, utc_all )
printUtc( 1, utc_all )
printUtc( -1, utc_all )

0.0 : 6.0 : 56.525
11.0 : 52.0 : 32.010
12.0 : 35.0 : 46.905


## logEkfEuler

### Load data

In [38]:
idx_timeStamp = 0
idx_roll = 1
idx_pitch = 2
idx_yaw = 3
idx_rollStdDev = 4
idx_pitchStdDev = 5
idx_yawStdDev = 6
idx_status = 7

In [39]:
euler = []
for idx in range(nbHours):
    euler.append( np.loadtxt( logEkfEuler[idx], skiprows = 1,
                             usecols = (0, idx_roll, idx_pitch, idx_yaw) ) )

In [40]:
euler_all = euler[0]
for idx in range(1, nbHours):
    euler_all = np.concatenate( (euler_all, euler[idx]), axis=0 )

In [41]:
eulerTimestamps = euler_all[:,0]
roll = euler_all[:,1]
pitch = euler_all[:,2]
yaw = euler_all[:,3]

### Shift timestamps

In [42]:
for k in range(1,eulerTimestamps.shape[0]):
    if eulerTimestamps[k] < eulerTimestamps[k-1]:
        print("k = {}".format(k))

In [43]:
eulerNewTimestamps = eulerTimestamps

## logEkfNav

### Load data

In [44]:
idx_timeStamp = 0
idx_velNorth = 1
idx_velEast = 2
idx_velDown = 3
idx_velNorth_StdDev = 4
idx_velEast_StdDev = 5
idx_velDown_StdDev = 6
idx_ekf_Lat = 7
idx_ekf_Long = 8
idx_ekf_Alt = 9
idx_ekf_undulation = 10
idx_ekf_Lat_StdDev = 11
idx_ekf_Long_StdDev = 12
idx_ekf_Alt_StdDev = 13
idx_status = 14

In [45]:
nav = []
for idx in range(nbHours):
    nav.append( np.loadtxt( logEkfNav[idx], skiprows = 1,
                           usecols = (0, idx_ekf_Lat, idx_ekf_Long, idx_ekf_Alt, idx_ekf_undulation, idx_status) ) )

In [46]:
nav_all = nav[0]
for idx in range(1, nbHours):
    nav_all = np.concatenate( (nav_all, nav[idx]), axis=0 )

In [47]:
navTimestamps = nav_all[:,0]
nav_Lat = nav_all[:,1]
nav_Long = nav_all[:,2]
nav_Alt = nav_all[:,3]
nav_Undulation = nav_all[:,4]
nav_status = nav_all[:,5].astype("uint32")

In [48]:
solution_mode = np.zeros( nav_status.shape )
for idx, val in enumerate(nav_status):
    solution_mode[idx] = val & 0b1111

### Shift timestamps

In [49]:
for k in range(1,navTimestamps.shape[0]):
    if navTimestamps[k] < navTimestamps[k-1]:
        print("k = {}".format(k))

In [50]:
navNewTimestamps = navTimestamps

### Transform EKF data

In [51]:
nav_proj = epsg.wgs84ToEpsg( (nav_Long, nav_Lat), epsg3xxx )
np.save( root_dir + "nav_epsg_transform", nav_proj )

# PoSAR-MC data

## Parameters

In [52]:
dates = ["14_12_21", "14_14_43", "14_18_28", "14_21_00"]

data_date = [ f"{day}_" + d for d in dates ]
data_dir = [root_dir +  d for d in data_date]

firstRecord = 0
lastRecord = [116, 110, 92, 126]

## Process timeStamps.data

In [53]:
def checkTimestamps(idx, dropFirst=0, withPlot=1):
    
    #====================================================================
    # the last value of the file may be erroneous, this should be checked
    #====================================================================

    #===================================================================
    # the first value may be erroneous, one may have to shift all values
    #===================================================================
    
    timeStampsFile = data_dir[idx] + "/" + data_date[idx] + "_timeStamps.data"
    print(timeStampsFile)
    bufferNumber, timeStamp = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

    if dropFirst:
        bufferNumber = bufferNumber[:-1]
        timeStamp = timeStamp[1:]
    
    if withPlot:
        plt.figure()
        plt.subplot(211)
        plt.plot( timeStamp, ".", label=data_date[idx] )
        plt.grid()
        plt.legend()
        plt.title("timeStamp wrt fileNumber / " + data_date[idx] )
        plt.subplot(212)
        plt.plot( np.diff(timeStamp)/1e6, "." )
        plt.grid()
        plt.title( "diff(timeStamp) wrt fileNumber / " + data_date[idx] )
    
    return bufferNumber, timeStamp

In [54]:
bufferNumber = {}
timeStamp = {}

In [55]:
rec = 0
dropFirst=0
withPlot=0
bufferNumber[rec], timeStamp[rec] = checkTimestamps(rec, dropFirst, withPlot)

/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/2019_07_12_14_12_21/2019_07_12_14_12_21_timeStamps.data


In [56]:
rec = 1
dropFirst=0
withPlot=0
bufferNumber[rec], timeStamp[rec] = checkTimestamps(rec, dropFirst, withPlot)

/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/2019_07_12_14_14_43/2019_07_12_14_14_43_timeStamps.data


In [57]:
rec = 2
dropFirst=0
withPlot=0
bufferNumber[rec], timeStamp[rec] = checkTimestamps(rec, dropFirst, withPlot)

/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/2019_07_12_14_18_28/2019_07_12_14_18_28_timeStamps.data


In [58]:
rec = 3
dropFirst=0
withPlot=0
bufferNumber[rec], timeStamp[rec] = checkTimestamps(rec, dropFirst, withPlot)

/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/2019_07_12_14_21_00/2019_07_12_14_21_00_timeStamps.data


**Be carefull here, there could be a jump in the timeStamp values due to the counter saturation at 2^32. The mitigation is following.**

In [59]:
timestampsShift = {}
timestampsShift[0] = 0 # this is for the SBG internal us loopback
timestampsShift[1] = 0
timestampsShift[2] = 0
timestampsShift[3] = 0

In [60]:
newTimeStamp = {}
for rec in timeStamp:
    newTimeStamp[rec] = timeStamp[rec] + timestampsShift[rec] * 2**32

In [61]:
for rec in newTimeStamp:
    printUtc( np.where(utcNewTimestamps < newTimeStamp[rec][0])[0][-1], utc_all )

12.0 : 12.0 : 12.715
12.0 : 14.0 : 32.640
12.0 : 18.0 : 19.205
12.0 : 20.0 : 49.440


## Selected record

In [103]:
selectedRecord = 1

In [104]:
idxTmp = np.where(event_all == timeStamp[selectedRecord][0])

### Read parameters

In [105]:
params_filename = data_dir[selectedRecord] + "/" + data_date[selectedRecord] + "_parameters.xml"
params = PosarXParameters( params_filename )

In [106]:
def getNbRecords( first, last, params ):
    nbRecords = [int((l - first) / params.buffersPerFile + 1) for l in last]
    return nbRecords

In [107]:
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
rampsPerBuffer = params.rampsPerBuffer
samplesPerFile = params.samplesPerRamp * params.rampsPerFile
buffersPerFile = params.buffersPerFile
T_files = samplesPerFile / 10e6

nbRecords = getNbRecords(firstRecord, lastRecord, params)

print( "nbRecords = {}".format(nbRecords) )
print( f"selectedRecord {selectedRecord} => {nbRecords[selectedRecord]} files" )

nbRecords = [59, 56, 47, 64]
selectedRecord 1 => 56 files


## Data interpolation

In [108]:
selectedTimeStamp = newTimeStamp[selectedRecord]
selectedBufferNumber = bufferNumber[selectedRecord]

### logEkfEuler

In [109]:
# interpolate euler data for all timeStamps
roll_selection = np.interp( selectedTimeStamp, eulerNewTimestamps, roll )
pitch_selection = np.interp( selectedTimeStamp, eulerNewTimestamps, pitch )
yaw_selection = np.interp( selectedTimeStamp, eulerNewTimestamps, yaw )

### logGpsPos

In [110]:
velNewTimestamps = gpsNewTimestamps

def getInterpolatedGps( timeStamp ):
    Lat_records    = np.interp( timeStamp, gpsNewTimestamps, Lat )
    Long_records   = np.interp( timeStamp, gpsNewTimestamps, Long )
    Alt_records    = np.interp( timeStamp, gpsNewTimestamps, Alt )
    Vel_records    = np.interp( timeStamp, velNewTimestamps, Vel )
    course_records = np.interp( timeStamp, velNewTimestamps, course )
    return Lat_records, Long_records, Alt_records, Vel_records, course_records

### Transform selected data

In [111]:
Lat_Long_Alt_Vel_course = getInterpolatedGps( selectedTimeStamp )
selectedEpsg = epsg.wgs84ToEpsg( (Lat_Long_Alt_Vel_course[1], Lat_Long_Alt_Vel_course[0]), epsg3xxx )

### logGpsVel

In [112]:
velNewTimestamps = gpsNewTimestamps

def getInterpolatedVel( timeStamp ):
    velNorth_records = np.interp( timeStamp, gpsNewTimestamps, velNorth )
    velEast_records  = np.interp( timeStamp, gpsNewTimestamps, velEast )
    velDown_records  = np.interp( timeStamp, gpsNewTimestamps, velDown )
    return velNorth_records, velEast_records, velDown_records

In [113]:
Vel_records = getInterpolatedVel( selectedTimeStamp )

## Plot navigation data and record periods

In [114]:
files = "files {} to {}".format( firstRecord, nbRecords[selectedRecord] )

In [115]:
fig, ax = plt.subplots(1,1)

for idx in range(nbHours):
    ax.plot( x[idx], y[idx], label=hours[idx] )
    
addRemarkablePoints( ax )
    
ax.plot( selectedEpsg[0], selectedEpsg[1], ".r", markeredgecolor='black', label=data_date[selectedRecord] )
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.grid()
ax.legend()
title = epsgStr + " " + day
ax.set_title(title)
#ax.set_aspect('equal')
fig.savefig( f"{data_dir[selectedRecord]}/{title}.png", bbox_inches='tight')

# Track approximation

## Orientation vector

In [ ]:
# East-West s0  => s1  14_12_21
# West-East s2  => s3  14_14_43
# South-North s4 => s5 14_18_28
# North-South s6 => s7 14_21_00
xA, yA = s6_epsg
xB, yB = s7_epsg
JA = s6
JB = s7

In [ ]:
xAB = (xB - xA) / ( (xB - xA)**2 + (yB - yA)**2 )**0.5
yAB = (yB - yA) / ( (xB - xA)**2 + (yB - yA)**2 )**0.5

In [ ]:
# unit vectors
ux = ( xAB, yAB )
uy = ( -yAB, xAB )

In [ ]:
# build sceneReferencePoint
JA_epsg = epsg.wgs84ToEpsg( (JA[1], JA[0]), epsg3xxx )
JB_epsg = epsg.wgs84ToEpsg( (JB[1], JB[0]), epsg3xxx )
JAJB = ( (JA_epsg[0]+JB_epsg[0])/2,
        (JA_epsg[1]+JB_epsg[1])/2 )
shiftInY = 0
sceneReferencePoint = JAJB[0] + shiftInY * uy[0], JAJB[1] + shiftInY * uy[1]

In [ ]:
import json

In [ ]:
track_model = { "trackModel" : "model(x, p) = p[1]*x + p[2]",
               "ux" : (ux[0], ux[1]),
               "uy" : (uy[0], uy[1]),
               "origX" : sceneReferencePoint[0],
               "origY" : sceneReferencePoint[1]
              }

In [ ]:
track_model

In [ ]:
trackFilename = data_dir[selectedRecord] + "/track_model.json"
with open( trackFilename, 'w' ) as f:
    json.dump( track_model, f )

In [ ]:
x_selection = np.interp( selectedTimeStamp, gpsNewTimestamps, gps_proj[0] )
y_selection = np.interp( selectedTimeStamp, gpsNewTimestamps, gps_proj[1] )
np.save( data_dir[selectedRecord] + "/track_selection_proj", np.stack( (x_selection, y_selection ), -1 ) )

# Interpolate navigation data for all ramps

In [ ]:
# rampNumber timeStamp x y z
nbRecordToAdd = 10

rampNumber = np.arange( rampsPerFile * ( nbRecords[selectedRecord] + nbRecordToAdd ) )

rampNumber_ext = np.arange(nbRecords[selectedRecord] + nbRecordToAdd) * rampsPerFile

lastDelta = selectedTimeStamp[-1] - selectedTimeStamp[-2]
# there is a difference between the number of valid timestamps and the number of records
diffValidTimestampsNbRecords = selectedTimeStamp.size - nbRecords[selectedRecord]
addedTimestamps = np.arange( 1, nbRecordToAdd - diffValidTimestampsNbRecords + 1 ) * lastDelta
print("lastDelta = {}".format(lastDelta) )
timeStamp_ext_a = np.concatenate( (selectedTimeStamp, 
                                 selectedTimeStamp[-1] + addedTimestamps ) )

In [ ]:
timestampsAllRamps = np.interp( rampNumber, rampNumber_ext, timeStamp_ext_a)

## logGpsPos (a)
Two options: numpy.interp vs scipy.interpolate.interp1d (with kind='linear')

### xyz

In [ ]:
gps_x = np.interp( timestampsAllRamps, gpsNewTimestamps, gps_proj[0] )
gps_y = np.interp( timestampsAllRamps, gpsNewTimestamps, gps_proj[1] )
gps_z = np.interp( timestampsAllRamps, gpsNewTimestamps, Alt )
xyz_proj_allRamps = np.stack( (rampNumber, timestampsAllRamps, 
                               gps_x, gps_y, gps_z), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_xyz_gps_a", xyz_proj_allRamps )

### lla latitude longitude altitude

In [ ]:
allRamps_lat = np.interp( timestampsAllRamps, gpsNewTimestamps, Lat )
allRamps_long = np.interp( timestampsAllRamps, gpsNewTimestamps, Long )
allRamps_alt = np.interp( timestampsAllRamps, gpsNewTimestamps, Alt )
latLongAlt_allRamps = np.stack( (rampNumber, timestampsAllRamps,
                                 allRamps_lat, allRamps_long, allRamps_alt), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_latLongAlt_gps_a", latLongAlt_allRamps )

## logEkfNav (a)

In [ ]:
nav_x = np.interp( timestampsAllRamps, navNewTimestamps, nav_proj[0] )
nav_y = np.interp( timestampsAllRamps, navNewTimestamps, nav_proj[1] )
nav_z = np.interp( timestampsAllRamps, navNewTimestamps, nav_Alt )
nav_u = np.interp( timestampsAllRamps, navNewTimestamps, nav_Undulation )
nav_xyz_allRamps = np.stack( (rampNumber, timestampsAllRamps, 
                              nav_x, nav_y, nav_z), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_xyz_nav_a", nav_xyz_allRamps )

In [ ]:
nav_allRamps_lat = np.interp( timestampsAllRamps, navNewTimestamps, nav_Lat )
nav_allRamps_long = np.interp( timestampsAllRamps, navNewTimestamps, nav_Long )
nav_allRamps_alt = np.interp( timestampsAllRamps, navNewTimestamps, nav_Alt )
nav_latLongAlt_allRamps = np.stack( (rampNumber, timestampsAllRamps, 
                              nav_allRamps_lat, nav_allRamps_long, nav_allRamps_alt), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_latLongAlt_nav_a", nav_latLongAlt_allRamps )

## logGpsPos (b)

In [ ]:
# rampNumber timeStamp x y z

timeStamp_ext_b = timeStamp_ext_a + params.rampPeriod / 2

timeStamp = np.interp( rampNumber, rampNumber_ext, timeStamp_ext_b)
x = np.interp( timeStamp, gpsNewTimestamps, gps_proj[0] )
y = np.interp( timeStamp, gpsNewTimestamps, gps_proj[1] )
z = np.interp( timeStamp, gpsNewTimestamps, Alt )
xyz_proj_allRamps = np.stack( (rampNumber, timeStamp, x, y, z), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_xyz_b", xyz_proj_allRamps )